In [ ]:
#Create a timeseries model that can predict concentrations of Pm2.5 in the atmosphere

In [2]:
## imports
import os
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import sklearn as skl
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
#path = os.path.join(os.getcwd(), 'Resources', 'all_city_data.csv')

In [5]:
df = pd.read_csv('all_city_data.csv').drop(columns = ['Unnamed: 0'])

In [6]:
df.head()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,dt
0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,2013-01-03 00:00:00
1,9.0,9.0,3.0,17.0,300.0,89.0,-0.5,1024.5,-21.4,0.0,2013-01-03 00:00:00
2,4.0,4.0,14.0,20.0,300.0,69.0,-0.7,1023.0,-18.8,0.0,2013-01-03 00:00:00
3,7.0,7.0,3.0,2.0,100.0,91.0,-2.3,1020.3,-20.7,0.0,2013-01-03 00:00:00
4,5.0,14.0,4.0,12.0,200.0,85.0,-0.5,1024.5,-21.4,0.0,2013-01-03 00:00:00


In [ ]:
# convert an array of values into a dataset matrix
# def create_dataset(dataset, look_back=1):
# 	dataX, dataY = [], []
# 	for i in range(len(dataset)-look_back-1):
# 		a = dataset[i:(i+look_back), 0]
# 		dataX.append(a)
# 		dataY.append(dataset[i + look_back, 0])
# 	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
# look_back = 1
# TrainX, TrainY = create_dataset(train, look_back)
# testX, testY = create_dataset(test, look_back)

In [ ]:

# reshape input to be [samples, time steps, features]
# trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [7]:
features = ["PM10", "SO2", "NO2", "CO", "O3", "TEMP", "PRES", "DEWP", "RAIN"]
scalar = StandardScaler()
# the fit_transform ops returns a 2d numpy.array, we cast it to a pd.DataFrame
standardized_features = pd.DataFrame(scalar.fit_transform(df[features].copy()), columns = features)
old_shape = df.shape
# drop the unnormalized features from the dataframe
df.drop(features, axis = 1, inplace = True)
# join back the normalized features
df = pd.concat([df, standardized_features], axis= 1)
assert old_shape == df.shape, "something went wrong!"

In [ ]:
df.head()

,PM2.5,dt,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN
0,4.0,2013-01-03 00:00:00,-1.101254,-0.545907,-1.243556,-0.804110,0.348630,-1.242142,1.164563,-1.538262,-0.078906
1,9.0,2013-01-03 00:00:00,-1.046543,-0.592877,-0.958503,-0.804110,0.560363,-1.224639,1.308088,-1.726789,-0.078906
2,4.0,2013-01-03 00:00:00,-1.101254,-0.076203,-0.872987,-0.804110,0.207475,-1.242142,1.164563,-1.538262,-0.078906
3,7.0,2013-01-03 00:00:00,-1.068428,-0.592877,-1.386082,-0.976692,0.595652,-1.382164,0.906219,-1.676032,-0.078906
4,5.0,2013-01-03 00:00:00,-0.991832,-0.545907,-1.101029,-0.890401,0.489786,-1.224639,1.308088,-1.726789,-0.078906


In [8]:
time_index=pd.to_datetime(df['dt'])
df_kl=df.set_index(time_index)
df_kl.index

DatetimeIndex(['2013-01-03 00:00:00', '2013-01-03 00:00:00',
               '2013-01-03 00:00:00', '2013-01-03 00:00:00',
               '2013-01-03 00:00:00', '2013-01-03 00:00:00',
               '2013-01-03 00:00:00', '2013-01-03 00:00:00',
               '2013-01-03 00:00:00', '2013-01-03 00:00:00',
               ...
               '2017-12-02 23:00:00', '2017-12-02 23:00:00',
               '2017-12-02 23:00:00', '2017-12-02 23:00:00',
               '2017-12-02 23:00:00', '2017-12-02 23:00:00',
               '2017-12-02 23:00:00', '2017-12-02 23:00:00',
               '2017-12-02 23:00:00', '2017-12-02 23:00:00'],
              dtype='datetime64[ns]', name='dt', length=383586, freq=None)

In [9]:
df_kl=df_kl.drop('dt', axis=1)
y = df_kl['PM2.5']
X = df_kl.drop(columns= ['PM2.5'])
X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle=False)



In [3]:
loaded_model = tf.keras.models.load_model("final_model.h5")
loaded_model.build()
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 2,057
Trainable params: 2,057
Non-trainable params: 0
_________________________________________________________________


In [10]:
loaded_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
loaded_model.fit(X_train, y_train, epochs=200)

Epoch 1/200
8991/8991 [==============================] - 14s 2ms/step - loss: 427.0831 - mean_squared_error: 427.0831
Epoch 2/200
8991/8991 [==============================] - 14s 2ms/step - loss: 427.2117 - mean_squared_error: 427.2117
Epoch 3/200
8991/8991 [==============================] - 14s 2ms/step - loss: 426.6203 - mean_squared_error: 426.6203
Epoch 4/200
8991/8991 [==============================] - 13s 2ms/step - loss: 427.6392 - mean_squared_error: 427.6392
Epoch 5/200
8991/8991 [==============================] - 13s 2ms/step - loss: 426.5056 - mean_squared_error: 426.5056
Epoch 6/200
8991/8991 [==============================] - 13s 1ms/step - loss: 426.4326 - mean_squared_error: 426.4326
Epoch 7/200
8991/8991 [==============================] - 13s 1ms/step - loss: 426.2110 - mean_squared_error: 426.2110
Epoch 8/200
8991/8991 [==============================] - 14s 2ms/step - loss: 426.2109 - mean_squared_error: 426.2109
Epoch 9/200
8991/8991 [==============================] -

In [11]:
scores = loaded_model.evaluate(X_test, y_test, verbose=2)
print(loaded_model.metrics_names[0], scores)

2997/2997 - 3s - loss: 497.3417 - mean_squared_error: 497.3417
loss [497.34173583984375, 497.34173583984375]


In [12]:
loaded_model.save("content/final_model.h5")

In [ ]:
# pred_temp = loaded_model.predict(X_test)
# pred_temp

array([[-0.25250596],
       [-0.19487654],
       [-0.13481413],
       ...,
       [-0.29821187],
       [-0.3370747 ],
       [-0.3753718 ]], dtype=float32)

In [ ]:

model=tf.keras.models.Sequential()
layer_1=tf.keras.layers.Dense(units=32, activation="sigmoid", input_dim=9)
model.add(layer_1)
layer_2=tf.keras.layers.Dense(units=32, activation="relu", input_dim=9)
model.add(layer_2)
layer_3=tf.keras.layers.Dense(units=20, activation="sigmoid", input_dim=9)
model.add(layer_3)
#layer_4=tf.keras.layers.Dense(units=48, activation="sigmoid", input_dim=9)
#model.add(layer_4)
#layer_5=tf.keras.layers.Dense(units=36, activation="sigmoid", input_dim=9)
#model.add(layer_5)
# layer_6 =tf.keras.layers.Dense(units=32, activation="sigmoid", input_dim=9)
# model.add(layer_6)
output_layer=tf.keras.layers.Dense(units=1)
model.add(output_layer)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 2,057
Trainable params: 2,057
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
model.fit(X_train, y_train, epochs=200)

Epoch 1/200
8991/8991 [==============================] - 13s 1ms/step - loss: 6430.4575 - mean_squared_error: 6430.4575
Epoch 2/200
8991/8991 [==============================] - 12s 1ms/step - loss: 2024.2274 - mean_squared_error: 2024.2274
Epoch 3/200
8991/8991 [==============================] - 12s 1ms/step - loss: 1010.7581 - mean_squared_error: 1010.7581
Epoch 4/200
8991/8991 [==============================] - 13s 1ms/step - loss: 744.9083 - mean_squared_error: 744.9083
Epoch 5/200
8991/8991 [==============================] - 13s 1ms/step - loss: 649.7569 - mean_squared_error: 649.7569
Epoch 6/200
8991/8991 [==============================] - 13s 1ms/step - loss: 601.3518 - mean_squared_error: 601.3518
Epoch 7/200
8991/8991 [==============================] - 13s 1ms/step - loss: 572.7335 - mean_squared_error: 572.7335
Epoch 8/200
8991/8991 [==============================] - 12s 1ms/step - loss: 553.1414 - mean_squared_error: 553.1414
Epoch 9/200
8991/8991 [===========================

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=2)
print(model.metrics_names[0], scores)

2997/2997 - 2s - loss: 504.7751 - mean_squared_error: 504.7751
loss [504.77508544921875, 504.77508544921875]


In [ ]:
model.save("content/final_model.h5")

In [ ]:
def create_model(hp):
    model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=9))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    model.compile(loss="mean_squared_error", optimizer='adam', metrics=["mean_squared_error"])
    
    return model

In [ ]:
pip install keras-tuner --upgrade

In [ ]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="mean_squared_error",
    max_epochs=20,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [ ]:
tuner.search(X_train,y_train,epochs=20,validation_data=(X_test,y_test))

Trial 45 Complete [00h 06m 22s]
mean_squared_error: 12609.1201171875

Best mean_squared_error So Far: 12609.1201171875
Total elapsed time: 02h 16m 52s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 3,
 'num_layers': 3,
 'tuner/bracket': 0,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_0': 5,
 'units_1': 7,
 'units_2': 5,
 'units_3': 5,
 'units_4': 3,
 'units_5': 7}

In [ ]:
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2997/2997 - 3s - loss: 12568.2422 - mean_squared_error: 12568.2422
Loss: 12568.2421875, Accuracy: 12568.2421875
